In [42]:
from pathlib import Path
import numpy as np
import pandas as pd

def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
    here = Path.cwd().resolve()
    for candidate in [here, *here.parents]:
        if any((candidate / m).exists() for m in markers):
            return candidate
    return here

ROOT = find_project_root()
data_path = ROOT / "docs" / "07-base-vendas-HP-EliteBook.xlsx"
results_dir = ROOT / "results"
results_dir.mkdir(parents=True, exist_ok=True)

def _parse_number(x):
    """Converte strings pt-BR -> float (milhar '.' e decimal ','). Inválidos -> NaN."""
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float, np.integer, np.floating)):
        return float(x)
    s = str(x).strip().replace(".", "").replace(",", ".")
    try:
        return float(s)
    except Exception:
        return np.nan

def _fmt_int(x):
    if x is None or (isinstance(x, float) and (np.isnan(x) or np.isinf(x))):
        return "—"
    return f"{int(x):,}".replace(",", "X").replace(".", ",").replace("X", ".")

def _fmt_pct(p):
    if p is None or (isinstance(p, float) and (np.isnan(p) or np.isinf(p))):
        return "—"
    return f"{p:.2f}%".replace(".", ",")

df = pd.read_excel(data_path)
df.columns = [c.strip() for c in df.columns]

if "Unidades_Vendidas" in df.columns:
    df["Unidades_Vendidas"] = pd.to_numeric(df["Unidades_Vendidas"], errors="coerce").astype("Int64")

if "Preço_Unitário" in df.columns:
    df["Preço_Unitário"] = df["Preço_Unitário"].apply(_parse_number)

if "Receita" in df.columns:
    df["Receita"] = df["Receita"].apply(_parse_number)

if "Ano" in df.columns:
    df["Ano"] = pd.to_numeric(df["Ano"], errors="coerce").astype("Int64")

if "Mês" in df.columns:
    df["Mês"] = pd.to_numeric(df["Mês"], errors="coerce").astype("Int64")

if all(col in df.columns for col in ["Receita", "Unidades_Vendidas", "Preço_Unitário"]):
    mask_null = df["Receita"].isna()
    df.loc[mask_null, "Receita"] = (
        df.loc[mask_null, "Unidades_Vendidas"].astype("float") *
        df.loc[mask_null, "Preço_Unitário"].astype("float")
    )

if "Ano" in df.columns and "Mês" in df.columns:
    df["AnoMes"] = pd.to_datetime(
        {"year": pd.to_numeric(df["Ano"], errors="coerce"),
         "month": pd.to_numeric(df["Mês"], errors="coerce"),
         "day": 1},
        errors="coerce",
    )

full_path = results_dir / "01_dataset_full.csv"
df.to_csv(full_path, index=False, encoding="utf-8")
print(f"[OK] Dataset COMPLETO salvo em: {full_path}  (linhas={len(df):,})")

preview_path = results_dir / "01_dataset_preview.csv"
df.head(240).to_csv(preview_path, index=False, encoding="utf-8")

ts_cont_filled = None
ts_csv = None
if "AnoMes" in df.columns:
    ts = (
        df.groupby("AnoMes", dropna=True)
          .agg(
              receita=("Receita", "sum"),
              unidades=("Unidades_Vendidas", "sum")
          )
          .sort_index()
    )
    if not isinstance(ts.index, pd.DatetimeIndex):
        ts.index = pd.to_datetime(ts.index, errors="coerce")

    if len(ts):
        full_idx = pd.date_range(start=ts.index.min(), end=ts.index.max(), freq="MS")
        ts_cont = ts.reindex(full_idx)
        ts_cont["Ano"] = ts_cont.index.year
        ts_cont_filled = ts_cont.fillna(0.0)

        ts_csv = results_dir / "01_timeseries_todos_anos.csv"
        ts_cont_filled.to_csv(ts_csv, index_label="AnoMes", encoding="utf-8")
        print(f"[OK] Timeseries contínua salva em: {ts_csv}  (meses={len(ts_cont_filled):,})")
else:
    print("[WARN] Coluna 'AnoMes' ausente — não foi possível gerar série contínua.")

por_ano_csv = None
if isinstance(ts_cont_filled, pd.DataFrame):
    por_ano = (
        ts_cont_filled
        .groupby("Ano", dropna=False, as_index=False)
        .agg(receita=("receita", "sum"), unidades=("unidades", "sum"))
        .sort_values("Ano")
    )
    por_ano_csv = results_dir / "01_resumo_por_ano.csv"
    por_ano.to_csv(por_ano_csv, index=False, encoding="utf-8")
    print(f"[OK] Resumo por ANO salvo em: {por_ano_csv}  (anos={por_ano['Ano'].nunique():,})")

print(f"[OK] Raiz do projeto: {ROOT}")
print(f"[OK] Lido de: {data_path}")
print(f"[OK] Resultados em: {results_dir}")
print(f"[OK] Prévia salva em: {preview_path}")
print(f"[INFO] Linhas: {len(df):,} | Colunas: {len(df.columns)}")

schema_rows = []
for col in df.columns:
    ser = df[col]
    dtype = str(ser.dtype)
    n_null = int(ser.isna().sum())
    pct_null = (n_null / len(df) * 100.0) if len(df) else np.nan
    nunique = int(ser.nunique(dropna=True))
    sample = ser.dropna().iloc[0] if ser.dropna().shape[0] else None
    row = {"coluna": col, "dtype": dtype, "nulos": n_null, "nulos_%": pct_null,
           "n_unique": nunique, "exemplo": sample}
    if pd.api.types.is_numeric_dtype(ser):
        s = pd.to_numeric(ser, errors="coerce")
        if s.notna().any():
            row.update({
                "min": float(np.nanmin(s)),
                "p25": float(np.nanpercentile(s.dropna(), 25)),
                "p50": float(np.nanpercentile(s.dropna(), 50)),
                "p75": float(np.nanpercentile(s.dropna(), 75)),
                "max": float(np.nanmax(s)),
            })
    schema_rows.append(row)

schema_df = pd.DataFrame(schema_rows).sort_values(["nulos_%", "coluna"], ascending=[False, True])
schema_path = results_dir / "01_schema_overview.csv"
schema_df.to_csv(schema_path, index=False, encoding="utf-8")

nulls_df = schema_df[["coluna", "nulos", "nulos_%"]].copy()
nulls_path = results_dir / "01_missing_by_col.csv"
nulls_df.to_csv(nulls_path, index=False, encoding="utf-8")

dup_count = int(df.duplicated().sum())
dup_sample_path = results_dir / "01_duplicates_sample.csv"
if dup_count > 0:
    df[df.duplicated(keep=False)].head(50).to_csv(dup_sample_path, index=False, encoding="utf-8")

periodo_txt = "—"
coverage_txt = "—"
if "AnoMes" in df.columns:
    amo = pd.to_datetime(df["AnoMes"], errors="coerce").dropna()
    if len(amo):
        start, end = amo.min(), amo.max()
        periodo_txt = f"{start.date()} — {end.date()}"
        full = pd.date_range(start=start, end=end, freq="MS")
        presentes = pd.DatetimeIndex(sorted(amo.unique()))
        missing = full.difference(presentes)
        coverage_txt = f"Meses no período: {len(full)} | Presentes: {len(presentes)} | Faltando: {len(missing)}"

style = """
<style>
body { font-family: system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, "Helvetica Neue", Arial; margin:24px; color:#111; }
h1, h2 { margin: 0 0 10px 0; }
h1 { font-size: 1.6rem; }
h2 { font-size: 1.2rem; color:#333; }
.grid { display:grid; grid-template-columns: repeat(auto-fill, minmax(260px, 1fr)); gap:14px; margin-top: 6px; }
.card { border:1px solid #e5e5e5; border-radius:14px; padding:14px 16px; box-shadow:0 1px 2px rgba(0,0,0,.03); }
.card h3 { margin:0 0 6px 0; font-size:0.95rem; color:#444; font-weight:600; }
.card p { margin:0; font-size:1.10rem; font-weight:700; letter-spacing:.2px; }
.note { background:#f7f7f7; border:1px solid #ececec; padding:12px; border-radius:10px; margin: 10px 0 18px 0; }
.small { color:#666; font-size: .92rem; }
.mono { font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; }
img { max-width:100%; height:auto; }
.tbl-scroll { max-height: 75vh; overflow: auto; border: 1px solid #eee; border-radius: 10px; }
.tbl-scroll table { border-collapse: collapse; width: 100%; font-size: 0.92rem; }
.tbl-scroll th, .tbl-scroll td { border: 1px solid #eee; padding: 6px 8px; text-align: left; vertical-align: top; }
.tbl-scroll th { position: sticky; top: 0; background: #fafafa; z-index: 1; }
</style>
"""

kpis = [
    ("📄 Linhas", _fmt_int(len(df))),
    ("📑 Colunas", _fmt_int(len(df.columns))),
    ("⛔ Células nulas", _fmt_int(int(df.isna().sum().sum()))),
    ("📦 Preview CSV", preview_path.name),
    ("📋 Esquema CSV", schema_path.name),
    ("⚠️ Nulos por coluna CSV", nulls_path.name),
    ("🔁 Duplicatas", _fmt_int(dup_count)),
    ("🗓️ Período (AnoMes)", periodo_txt),
    ("📆 Cobertura (AnoMes)", coverage_txt),
    ("📚 Dataset COMPLETO", full_path.name),
] + (
    [("📈 Timeseries (todos os meses/anos)", ts_csv.name)] if ts_csv else []
) + (
    [("🗓️ Resumo por ano", por_ano_csv.name)] if por_ano_csv else []
)

cards_html = "\n".join([f'<div class="card"><h3>{k}</h3><p>{v}</p></div>' for k, v in kpis])

schema_table = schema_df.copy()
schema_table["nulos_%"] = schema_table["nulos_%"].map(_fmt_pct)
schema_html = schema_table.to_html(index=False, escape=False)

full_table_html = df.to_html(index=False, escape=False)

report_path = results_dir / "01_data_report.html"
html = f"""<!doctype html>
<meta charset="utf-8">
<title>Data Report — Base de Vendas</title>
{style}
<h1>Data Report — Base de Vendas</h1>
<div class="note mono small">Arquivo: {data_path}</div>

<h2>Resumo</h2>
<div class="grid">{cards_html}</div>

<h2 style="margin-top:18px">Esquema & Qualidade (tabela completa)</h2>
<div class="small">Colunas, dtypes, nulos, cardinalidade e estatísticas (p25/50/75 para numéricos).</div>
{schema_html}

<h2 style="margin-top:18px">Tabela completa (todas as linhas)</h2>
<div class="small">Exibindo <strong>{len(df):,}</strong> linhas. Use o scroll; o cabeçalho permanece fixo.</div>
<div class="tbl-scroll">
{full_table_html}
</div>

<p class="small mono" style="margin-top:14px">
Arquivos: {full_path.name} • {preview_path.name} • {schema_path.name} • {nulls_path.name}
{" • " + dup_sample_path.name if dup_count>0 else ""}
{" • " + (ts_csv.name if ts_csv else "")}
{" • " + (por_ano_csv.name if por_ano_csv else "")}
</p>
"""
report_path.write_text(html, encoding="utf-8")

report_inline_path = results_dir / "01_data_report_inline.html"
report_inline_path.write_text(html, encoding="utf-8")

print(f"[OK] Relatório HTML: {report_path}")
print(f"[OK] Relatório HTML INLINE: {report_inline_path}")

df.head()

[OK] Dataset COMPLETO salvo em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\01_dataset_full.csv  (linhas=240)
[OK] Timeseries contínua salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\01_timeseries_todos_anos.csv  (meses=60)
[OK] Resumo por ANO salvo em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\01_resumo_por_ano.csv  (anos=5)
[OK] Raiz do projeto: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python
[OK] Lido de: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\docs\07-base-vendas-HP-EliteBook.xlsx
[OK] Resultados em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results
[OK] Prévia salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\01_dataset_preview.csv
[INFO] Linhas: 240 | Colunas: 16
[OK] Relatório HTML: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\01_data_report.html
[OK] Relatório HTML INLINE: C:\Users\Vinícius Andrade\D

,Ano,Mês,Produto,Canal,Regiões,Unidades_Vendidas,Preço_Unitário,Receita,Sazonalidade,Faixa Etária 25 a 40 anos,Faixa Etária 30 a 45 anos,Faixa Etária 30+ anos,Faixa Etária 35+ anos,Loja,Funcionario,AnoMes
0,2020,1,"EliteBook 840 G6 (Intel i5, 8GB, 256GB SSD)",Site,Norte,562,5200.0,2922400.0,NaN,169,197,112,84,Loja B,Bruna,2020-01-01
1,2020,1,"EliteBook 840 G8 (Intel i5, 16GB, 512GB SSD)",Site,Nordeste,339,6750.0,2288250.0,NaN,102,119,68,51,Loja B,Lucas,2020-01-01
2,2020,1,"EliteBook 640 G9 (Intel i7, 16GB, 512GB SSD)",Marketplace,Centro-Oeste,555,2300.0,1276500.0,NaN,167,194,111,83,Loja B,Lucas,2020-01-01
3,2020,1,"EliteBook x360 1040 G11 (Intel i7, 32GB, 1TB SSD)",Marketplace,Sudeste,528,9500.0,5016000.0,NaN,158,185,106,79,Loja C,Jéssica,2020-01-01
4,2020,2,"EliteBook 840 G6 (Intel i5, 8GB, 256GB SSD)",Marketplace,Sul,457,5200.0,2376400.0,NaN,137,160,91,69,Loja C,Isabela,2020-02-01


In [43]:
from pathlib import Path
import numpy as np
import pandas as pd

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here
    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

def _fmt_int(x):
    if x is None or (isinstance(x, float) and (np.isnan(x) or np.isinf(x))):
        return "—"
    return f"{int(x):,}".replace(",", "X").replace(".", ",").replace("X", ".")

n_rows, n_cols = df.shape

shape_info = pd.DataFrame(
    {"metric": ["rows", "cols"], "value": [int(n_rows), int(n_cols)]}
)

missing = df.isna().sum().reset_index()
missing.columns = ["coluna", "missing"]
if n_rows > 0:
    missing["missing_%"] = missing["missing"].astype(float).div(n_rows).mul(100.0)
else:
    missing["missing_%"] = np.nan

duplicates_count = int(df.duplicated().sum())

unique_counts = df.nunique(dropna=True).reset_index()
unique_counts.columns = ["coluna", "unique_values"]

shape_info.to_csv(results_dir / "02_shape_info.csv", index=False, encoding="utf-8")
missing.to_csv(results_dir / "02_missing_values.csv", index=False, encoding="utf-8")
unique_counts.to_csv(results_dir / "02_unique_counts.csv", index=False, encoding="utf-8")

with (results_dir / "02_data_profile.txt").open("w", encoding="utf-8") as f:
    f.write(f"rows={n_rows} cols={n_cols}\n")
    f.write(f"duplicates={duplicates_count}\n")

print(f"[OK] Perfil de qualidade salvo em: {results_dir}")

style = """
<style>
body { font-family: system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, "Helvetica Neue", Arial; margin:24px; color:#111; }
h1, h2 { margin: 0 0 10px 0; }
h1 { font-size: 1.6rem; }
h2 { font-size: 1.2rem; color:#333; }
.grid { display:grid; grid-template-columns: repeat(auto-fill, minmax(260px, 1fr)); gap:14px; margin-top: 6px; }
.card { border:1px solid #e5e5e5; border-radius:14px; padding:14px 16px; box-shadow:0 1px 2px rgba(0,0,0,.03); }
.card h3 { margin:0 0 6px 0; font-size:0.95rem; color:#444; font-weight:600; }
.card p { margin:0; font-size:1.10rem; font-weight:700; letter-spacing:.2px; }
.small { color:#666; font-size:.92rem; }
.mono { font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; }
.tbl-scroll { max-height: 72vh; overflow:auto; border:1px solid #eee; border-radius:10px; }
.tbl-scroll table { border-collapse:collapse; width:100%; font-size:.92rem; }
.tbl-scroll th, .tbl-scroll td { border:1px solid #eee; padding:6px 8px; text-align:left; vertical-align:top; }
.tbl-scroll th { position:sticky; top:0; background:#fafafa; z-index:1; }
</style>
"""

cards = [
    ("📄 Linhas", _fmt_int(n_rows)),
    ("📑 Colunas", _fmt_int(n_cols)),
    ("🔁 Duplicatas", _fmt_int(duplicates_count)),
    ("📦 02_shape_info.csv", "gerado"),
    ("⚠️ 02_missing_values.csv", "gerado"),
    ("🔣 02_unique_counts.csv", "gerado"),
    ("📝 02_data_profile.txt", "gerado"),
]
cards_html = "\n".join([f'<div class="card"><h3>{k}</h3><p>{v}</p></div>' for k, v in cards])

miss_tbl = (missing
            .assign(missing_pct=lambda d: d["missing_%"].round(2))
            .sort_values(["missing_%","missing"], ascending=[False, False])
            .rename(columns={"missing_%":"missing_% (float)", "missing_pct":"missing_% (arred.)"})
            .to_html(index=False, escape=False))

uniq_tbl = (unique_counts
            .sort_values("unique_values", ascending=False)
            .to_html(index=False, escape=False))

html = f"""<!doctype html>
<meta charset="utf-8">
<title>02 · Perfil de qualidade</title>
{style}
<h1>02 · Perfil de qualidade (dataset)</h1>

<h2>Resumo</h2>
<div class="grid">{cards_html}</div>

<h2 style="margin-top:18px">Nulos por coluna</h2>
<div class="small">Contagem e porcentagem de valores ausentes por coluna (ordenado por %).</div>
<div class="tbl-scroll">{miss_tbl}</div>

<h2 style="margin-top:18px">Cardinalidade por coluna</h2>
<div class="small">Número de valores distintos por coluna (exclui NA por padrão).</div>
<div class="tbl-scroll">{uniq_tbl}</div>

<p class="small mono" style="margin-top:14px">
Arquivos gerados: 02_shape_info.csv • 02_missing_values.csv • 02_unique_counts.csv • 02_data_profile.txt
</p>
"""

report02 = results_dir / "02_quality_profile.html"
report02.write_text(html, encoding="utf-8")
print(f"[OK] Relatório HTML (02): {report02}")

shape_info

[OK] Perfil de qualidade salvo em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results
[OK] Relatório HTML (02): C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\02_quality_profile.html


,metric,value
0,rows,240
1,cols,16


In [44]:
import math
from matplotlib.dates import MonthLocator, YearLocator, DateFormatter
from matplotlib.lines import Line2D

periodo_txt, chart_ok = "—", False
chart_png = results_dir / "03_kpis_linha_receita.png"

if "AnoMes" in df.columns and "Receita" in df.columns:
    ts = (pd.DataFrame({"AnoMes": pd.to_datetime(df["AnoMes"], errors="coerce"),
                        "Receita": pd.to_numeric(df["Receita"], errors="coerce")})
            .dropna(subset=["AnoMes"])
            .groupby("AnoMes", as_index=False).agg(Receita=("Receita","sum"))
            .sort_values("AnoMes"))

    if len(ts):
        start, end = ts["AnoMes"].min(), ts["AnoMes"].max()
        periodo_txt = f"{start.date()} — {end.date()}"

        x = ts["AnoMes"].to_numpy()
        y = ts["Receita"].to_numpy(float)
        y_ma3 = pd.Series(y).rolling(3, min_periods=1).mean().to_numpy()

        y_min, y_q1, y_med, y_q3, y_max = np.nanpercentile(y, [0, 25, 50, 75, 100])
        levels = [y_min, y_q1, y_med, y_q3, y_max]
        names  = ["mín", "Q1", "mediana", "Q3", "máx"]
        y_levels, y_labels = [], []
        for val, nm in zip(levels, names):
            if not y_levels or not np.isclose(val, y_levels[-1], atol=1e-9):
                y_levels.append(float(val))
                y_labels.append(f"{brl_full(val)} ({nm})")

        fig, ax = plt.subplots(figsize=(11.5, 4.1), dpi=170)

        ln1, = ax.plot(x, y, lw=2, label="Receita mensal")
        ln2, = ax.plot(x, y_ma3, lw=1.6, ls="--", label="Média móvel (3m)")
        ax.fill_between(x, y, min(y_min, y_med), alpha=0.10)

        months_span = (end.year - start.year) * 12 + (end.month - start.month) + 1
        interval = max(1, math.ceil(months_span / 8))
        ax.xaxis.set_major_locator(MonthLocator(interval=interval))
        ax.xaxis.set_major_formatter(DateFormatter("%b/%Y"))
        ax.xaxis.set_minor_locator(YearLocator())
        ax.tick_params(axis="x", which="major", labelsize=9, pad=6)
        ax.grid(True, axis="x", which="minor", alpha=0.10)
        ax.set_xlabel("Tempo (meses)")
        ax.set_xlim(x[0], x[-1])

        ax.set_ylabel("Receita (BRL)")
        ax.set_yticks(y_levels)
        ax.set_yticklabels(y_labels)
        ax.yaxis.set_major_formatter(mticker.FuncFormatter(brl_abbrev))
        for lvl in y_levels:
            ax.axhline(lvl, color="#9ca3af", lw=1, alpha=0.25, zorder=0)
        pad = (y_max - y_min) * 0.06 if y_max > y_min else 1.0
        ax.set_ylim(y_min - pad, y_max + pad)

        i_min, i_max = int(np.argmin(y)), int(np.argmax(y))
        ax.scatter([x[i_min], x[i_max], x[-1]], [y[i_min], y[i_max], y[-1]],
                   s=30, color="#111827", zorder=3)
        ax.annotate(f"Mín: {brl_full(y[i_min])}", xy=(x[i_min], y[i_min]),
                    xytext=(10, -12), textcoords="offset points",
                    ha="left", va="top", fontsize=8,
                    bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="0.8"))
        ax.annotate(f"Máx: {brl_full(y[i_max])}", xy=(x[i_max], y[i_max]),
                    xytext=(10, 12), textcoords="offset points",
                    ha="left", va="bottom", fontsize=8,
                    bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="0.8"))
        ax.annotate(f"Último: {brl_full(y[-1])}", xy=(x[-1], y[-1]),
                    xytext=(10, 0), textcoords="offset points",
                    ha="left", va="center", fontsize=8,
                    bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="0.8"))

        dot = Line2D([], [], marker='o', linestyle='None', color="#111827", label="Mín/Máx/Último")
        ax.legend(handles=[ln1, ln2, dot], title="Séries",
                  loc="lower left", bbox_to_anchor=(0, 1.02),
                  ncol=3, frameon=True, framealpha=0.95,
                  facecolor="white", edgecolor="#e5e7eb")

        ax.grid(True, axis="y", alpha=0.25)
        plt.tight_layout(rect=[0, 0, 1, 0.90])
        fig.savefig(chart_png, bbox_inches="tight")
        plt.close(fig)
        chart_ok = True

In [45]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here


    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

assert "df" in locals(), "DataFrame 'df' não encontrado."
if "AnoMes" not in df.columns:
    if {"Ano", "Mês"} <= set(df.columns):
        df["AnoMes"] = pd.to_datetime(
            {"year": pd.to_numeric(df["Ano"], errors="coerce"),
             "month": pd.to_numeric(df["Mês"], errors="coerce"),
             "day": 1},
            errors="coerce",
        )
    else:
        raise RuntimeError("Faltam 'AnoMes' ou ('Ano','Mês') para construir a série temporal.")

df = df.dropna(subset=["AnoMes"])
if not pd.api.types.is_datetime64_any_dtype(df["AnoMes"]):
    df["AnoMes"] = pd.to_datetime(df["AnoMes"], errors="coerce")
df = df.dropna(subset=["AnoMes"])

ts = (
    df.groupby("AnoMes", dropna=True, as_index=False)
    .agg(receita=("Receita", "sum"),
         unidades=("Unidades_Vendidas", "sum"))
    .sort_values("AnoMes")
    .set_index("AnoMes")
)

ts.index = pd.to_datetime(ts.index, errors="coerce")
ts = ts.resample("MS").sum()

ts["receita_ma3"] = ts["receita"].rolling(window=3, min_periods=1).mean()
ts["unidades_ma3"] = ts["unidades"].rolling(window=3, min_periods=1).mean()


def brl_abbrev(x, pos):
    ax = abs(x)
    if ax >= 1e9:
        v, suf = x / 1e9, " bi"
    elif ax >= 1e6:
        v, suf = x / 1e6, " mi"
    elif ax >= 1e3:
        v, suf = x / 1e3, " mil"
    else:
        v, suf = x, ""
    s = f"R$ {v:,.1f}".replace(",", "X").replace(".", ",").replace("X", ".")
    return s + suf


def int_abbrev(x, pos):
    ax = abs(x)
    if ax >= 1e6:  return f"{x / 1e6:.1f}M"
    if ax >= 1e3:  return f"{x / 1e3:.1f}k"
    return f"{x:.0f}"


nmeses = max(1, len(ts.index))
if nmeses <= 24:
    major = mdates.MonthLocator(interval=1)
elif nmeses <= 48:
    major = mdates.MonthLocator(interval=3)
elif nmeses <= 84:
    major = mdates.MonthLocator(interval=6)
else:
    major = mdates.YearLocator()
date_fmt = mdates.DateFormatter("%Y-%m")

fig1, ax1 = plt.subplots(figsize=(12, 5))
ax1.plot(ts.index, ts["receita"], label="Receita mensal")
ax1.plot(ts.index, ts["receita_ma3"], linestyle="--", label="Média móvel 3m")
ax1.set_title("Receita por mês (soma) — com média móvel 3m")
ax1.set_xlabel("Ano-Mês")
ax1.set_ylabel("Receita (BRL)")
ax1.xaxis.set_major_locator(major)
ax1.xaxis.set_major_formatter(date_fmt)
for lbl in ax1.get_xticklabels(): lbl.set_rotation(45); lbl.set_ha("right")
ax1.yaxis.set_major_formatter(mticker.FuncFormatter(brl_abbrev))
ax1.grid(True, axis="y", alpha=0.3)
ax1.legend()
plt.tight_layout()

if ts["receita"].notna().any():
    t_max, y_max = ts["receita"].idxmax(), ts["receita"].max()
    t_min, y_min = ts["receita"].idxmin(), ts["receita"].min()
    ax1.annotate("pico", xy=(t_max, y_max), xytext=(0, 10), textcoords="offset points")
    ax1.annotate("vale", xy=(t_min, y_min), xytext=(0, -15), textcoords="offset points")

fig1_svg = results_dir / "04_timeseries_receita.svg"
fig1_png = results_dir / "04_timeseries_receita.png"
fig1.savefig(fig1_svg, bbox_inches="tight")
fig1.savefig(fig1_png, dpi=160, bbox_inches="tight")
plt.close(fig1)

fig2, ax2 = plt.subplots(figsize=(12, 5))
ax2.plot(ts.index, ts["unidades"], label="Unidades/mês")
ax2.plot(ts.index, ts["unidades_ma3"], linestyle="--", label="Média móvel 3m")
ax2.set_title("Unidades vendidas por mês — com média móvel 3m")
ax2.set_xlabel("Ano-Mês")
ax2.set_ylabel("Unidades")
ax2.xaxis.set_major_locator(major)
ax2.xaxis.set_major_formatter(date_fmt)
for lbl in ax2.get_xticklabels(): lbl.set_rotation(45); lbl.set_ha("right")
ax2.yaxis.set_major_formatter(mticker.FuncFormatter(int_abbrev))
ax2.grid(True, axis="y", alpha=0.3)
ax2.legend()
plt.tight_layout()

if ts["unidades"].notna().any():
    t_max, y_max = ts["unidades"].idxmax(), ts["unidades"].max()
    t_min, y_min = ts["unidades"].idxmin(), ts["unidades"].min()
    ax2.annotate("pico", xy=(t_max, y_max), xytext=(0, 10), textcoords="offset points")
    ax2.annotate("vale", xy=(t_min, y_min), xytext=(0, -15), textcoords="offset points")

fig2_svg = results_dir / "04_timeseries_unidades.svg"
fig2_png = results_dir / "04_timeseries_unidades.png"
fig2.savefig(fig2_svg, bbox_inches="tight")
fig2.savefig(fig2_png, dpi=160, bbox_inches="tight")
plt.close(fig2)

ts_out = ts.reset_index().rename(columns={"AnoMes": "ano_mes"})
ts_path = results_dir / "04_timeseries.csv"
ts_out.to_csv(ts_path, index=False, encoding="utf-8")

total_receita = float(ts["receita"].sum())
media_mensal_unid = float(ts["unidades"].mean())


def brl_full(x):
    s = f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    return s


summary_html = f"""
<h2>Resumo</h2>
<ul>
  <li>Meses analisados: <b>{nmeses}</b></li>
  <li>Receita total: <b>{brl_full(total_receita)}</b></li>
  <li>Unidades/mês (média): <b>{media_mensal_unid:,.0f}</b></li>
</ul>
"""

report_path = results_dir / "04_timeseries_report.html"
report_html = f"""<!doctype html>
<meta charset="utf-8">
<title>Timeseries — Receita e Unidades</title>
<style>
body {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, "Helvetica Neue", Arial; margin: 24px; }}
figure {{ margin: 0 0 28px 0; }}
figcaption {{ margin-top: 8px; font-size: 0.95rem; color: #444; }}
img {{ max-width: 100%; height: auto; }}
.code {{ font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, "Liberation Mono", monospace; }}
hr {{ border: none; border-top: 1px solid #ddd; margin: 28px 0; }}
</style>
<h1>Séries temporais — Receita e Unidades</h1>
{summary_html}
<hr/>
<figure>
  <img src="{fig1_svg.name}" alt="Receita por mês (com média móvel)">
  <figcaption>Receita mensal agregada e média móvel 3 meses.</figcaption>
</figure>
<figure>
  <img src="{fig2_svg.name}" alt="Unidades por mês (com média móvel)">
  <figcaption>Unidades mensais e média móvel 3 meses.</figcaption>
</figure>
<p class="code">CSV: {ts_path.name}</p>
"""
report_path.write_text(report_html, encoding="utf-8")

print(f"[OK] Série temporal salva em: {ts_path}")
print(f"[OK] Figuras vetoriais: {fig1_svg} ; {fig2_svg}")
print(f"[OK] Relatório HTML: {report_path}")

[OK] Série temporal salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\04_timeseries.csv
[OK] Figuras vetoriais: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\04_timeseries_receita.svg ; C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\04_timeseries_unidades.svg
[OK] Relatório HTML: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\04_timeseries_report.html


In [46]:
from pathlib import Path
import io, base64
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here


    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

assert "df" in locals(), "DataFrame 'df' não encontrado."
assert "Produto" in df.columns, "Coluna 'Produto' ausente."


def brl_abbrev(x, pos=None):
    ax = abs(x)
    if ax >= 1e9:
        v, suf = x / 1e9, " bi"
    elif ax >= 1e6:
        v, suf = x / 1e6, " mi"
    elif ax >= 1e3:
        v, suf = x / 1e3, " mil"
    else:
        v, suf = x, ""
    s = f"R$ {v:,.1f}".replace(",", "X").replace(".", ",").replace("X", ".")
    return s + suf


def int_abbrev(x, pos=None):
    ax = abs(x)
    if ax >= 1e6:  return f"{x / 1e6:.1f}M"
    if ax >= 1e3:  return f"{x / 1e3:.1f}k"
    return f"{x:.0f}"


def brl_full(x: float) -> str:
    s = f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    return s


def _weighted_mean(values, weights):
    v = pd.Series(values, dtype="float")
    w = pd.Series(weights, dtype="float")
    num = (v * w).sum(min_count=1)
    den = w.sum(min_count=1)
    return (num / den) if den and not math.isnan(den) else float("nan")


prod = (
    df.groupby("Produto", dropna=True, as_index=False)
    .agg(
        receita=("Receita", "sum"),
        unidades=("Unidades_Vendidas", "sum"),
        preco_medio=("Preço_Unitário", "mean"),
    )
    .sort_values("receita", ascending=False)
)

total_receita = float(prod["receita"].sum()) if len(prod) else 0.0
prod["share_receita_%"] = (prod["receita"] / total_receita * 100.0) if total_receita else 0.0

prod_path = results_dir / "05_produto_desempenho.csv"
prod.to_csv(prod_path, index=False, encoding="utf-8")
print(f"[OK] Tabela produto salva em: {prod_path}")

TOP_N = 15
top = prod.nlargest(TOP_N, "receita").copy()

if len(prod) > TOP_N:
    resto = prod.iloc[TOP_N:]
    outros = pd.DataFrame({
        "Produto": ["Outros"],
        "receita": [resto["receita"].sum()],
        "unidades": [resto["unidades"].sum()],
        "preco_medio": [_weighted_mean(resto["preco_medio"], resto["unidades"])],
        "share_receita_%": [resto["share_receita_%"].sum()],
    })
    top_plot = pd.concat([top, outros], ignore_index=True)
else:
    top_plot = top

top_plot = top_plot.reset_index(drop=True)
top_plot = top_plot.sort_values("receita", ascending=True)

fig1, ax1 = plt.subplots(figsize=(12, max(6, 0.45 * len(top_plot))))
y = top_plot["Produto"]
x = top_plot["receita"]
bars1 = ax1.barh(y, x)
ax1.set_title("Receita por Produto — Top 15 (+ Outros)")
ax1.set_xlabel("Receita (BRL)")
ax1.xaxis.set_major_formatter(mticker.FuncFormatter(brl_abbrev))
ax1.grid(True, axis="x", alpha=0.25)

labels1 = [f"{brl_abbrev(v)}  •  {s:.1f}%" for v, s in zip(x, top_plot["share_receita_%"])]
try:
    ax1.bar_label(bars1, labels=labels1, padding=2)
except Exception:
    for rect, txt in zip(bars1, labels1):
        ax1.text(rect.get_width(), rect.get_y() + rect.get_height() / 2, " " + txt,
                 va="center", ha="left")
plt.tight_layout()
fig1_svg = results_dir / "05_produto_receita_top.svg"
fig1_png = results_dir / "05_produto_receita_top.png"
fig1.savefig(fig1_svg, bbox_inches="tight")
fig1.savefig(fig1_png, dpi=160, bbox_inches="tight")
plt.close(fig1)

fig2, ax2 = plt.subplots(figsize=(12, max(6, 0.45 * len(top_plot))))
y2 = top_plot["Produto"]
x2 = top_plot["unidades"]
bars2 = ax2.barh(y2, x2)
ax2.set_title("Unidades vendidas por Produto — Top 15 (+ Outros)")
ax2.set_xlabel("Unidades")
ax2.xaxis.set_major_formatter(mticker.FuncFormatter(int_abbrev))
ax2.grid(True, axis="x", alpha=0.25)
labels2 = [f"{int_abbrev(v)}" for v in x2]
try:
    ax2.bar_label(bars2, labels=labels2, padding=2)
except Exception:
    for rect, txt in zip(bars2, labels2):
        ax2.text(rect.get_width(), rect.get_y() + rect.get_height() / 2, " " + txt,
                 va="center", ha="left")
plt.tight_layout()
fig2_svg = results_dir / "05_produto_unidades_top.svg"
fig2_png = results_dir / "05_produto_unidades_top.png"
fig2.savefig(fig2_svg, bbox_inches="tight")
fig2.savefig(fig2_png, dpi=160, bbox_inches="tight")
plt.close(fig2)

print(f"[OK] Figuras: {fig1_svg} ; {fig1_png}")
print(f"[OK] Figuras: {fig2_svg} ; {fig2_png}")

share_top = top["share_receita_%"].sum()
report_path = results_dir / "05_produto_report.html"
report_html = f"""<!doctype html>
<meta charset="utf-8">
<title>Produtos — Desempenho</title>
<style>
body {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, "Helvetica Neue", Arial; margin: 24px; color: #111; }}
figure {{ margin: 0 0 28px 0; }}
figcaption {{ margin-top: 8px; font-size: 0.95rem; color: #444; }}
img {{ max-width: 100%; height: auto; border: 0; }}
p.code {{ font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; font-size: 0.95rem; }}
ul {{ line-height: 1.45; }}
hr {{ border: none; border-top: 1px solid #ddd; margin: 28px 0; }}
.note {{ background: #f6f6f6; border: 1px solid #e5e5e5; padding: 12px; border-radius: 8px; font-size: 0.95rem; }}
</style>

<h1>Desempenho por Produto</h1>
<div class="note">
  Abra este arquivo no mesmo diretório dos artefatos: <b>{fig1_svg.name}</b>, <b>{fig2_svg.name}</b> e <b>{prod_path.name}</b>.
</div>

<ul>
  <li>Total de produtos: <b>{len(prod)}</b></li>
  <li>Participação dos Top {min(TOP_N, len(prod))}: <b>{share_top:.1f}%</b> da receita</li>
  <li>Receita total: <b>{brl_full(total_receita)}</b></li>
</ul>

<figure>
  <img src="{fig1_svg.name}" alt="Receita por Produto — Top 15 (+ Outros)">
  <figcaption>Receita por produto com rótulos (valor + participação).</figcaption>
</figure>

<figure>
  <img src="{fig2_svg.name}" alt="Unidades por Produto — Top 15 (+ Outros)">
  <figcaption>Unidades vendidas por produto com rótulos.</figcaption>
</figure>

<p class="code">CSV: {prod_path.name}</p>
"""
report_path.write_text(report_html, encoding="utf-8")
print(f"[OK] Relatório HTML (arquivos externos): {report_path}")


def _encode_png_base64(png_path: Path) -> str:
    data = png_path.read_bytes()
    b64 = base64.b64encode(data).decode("ascii")
    return f"data:image/png;base64,{b64}"


img1_data = _encode_png_base64(fig1_png)
img2_data = _encode_png_base64(fig2_png)

report_inline_path = results_dir / "05_produto_report_inline.html"
report_inline_html = f"""<!doctype html>
<meta charset="utf-8">
<title>Produtos — Desempenho (Inline)</title>
<style>
body {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, "Helvetica Neue", Arial; margin: 24px; color: #111; }}
figure {{ margin: 0 0 28px 0; }}
figcaption {{ margin-top: 8px; font-size: 0.95rem; color: #444; }}
img {{ max-width: 100%; height: auto; border: 0; }}
p.code {{ font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; font-size: 0.95rem; }}
ul {{ line-height: 1.45; }}
hr {{ border: none; border-top: 1px solid #ddd; margin: 28px 0; }}
</style>

<h1>Desempenho por Produto — Relatório Inline</h1>
<ul>
  <li>Total de produtos: <b>{len(prod)}</b></li>
  <li>Participação dos Top {min(TOP_N, len(prod))}: <b>{share_top:.1f}%</b> da receita</li>
  <li>Receita total: <b>{brl_full(total_receita)}</b></li>
</ul>

<figure>
  <img src="{img1_data}" alt="Receita por Produto — Top 15 (+ Outros)">
  <figcaption>Receita por produto com rótulos (valor + participação).</figcaption>
</figure>

<figure>
  <img src="{img2_data}" alt="Unidades por Produto — Top 15 (+ Outros)">
  <figcaption>Unidades vendidas por produto com rótulos.</figcaption>
</figure>

<p class="code">CSV: {prod_path.name} (arquivo externo)</p>
"""
report_inline_path.write_text(report_inline_html, encoding="utf-8")
print(f"[OK] Relatório HTML INLINE (auto-contido): {report_inline_path}")

[OK] Tabela produto salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\05_produto_desempenho.csv
[OK] Figuras: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\05_produto_receita_top.svg ; C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\05_produto_receita_top.png
[OK] Figuras: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\05_produto_unidades_top.svg ; C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\05_produto_unidades_top.png
[OK] Relatório HTML (arquivos externos): C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\05_produto_report.html
[OK] Relatório HTML INLINE (auto-contido): C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\05_produto_report_inline.html


In [47]:
from pathlib import Path
import base64, math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here


    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

assert "df" in locals(), "DataFrame 'df' não encontrado."
assert "Canal" in df.columns, "Coluna 'Canal' ausente."


def brl_abbrev(x, pos=None):
    ax = abs(x)
    if ax >= 1e9:
        v, suf = x / 1e9, " bi"
    elif ax >= 1e6:
        v, suf = x / 1e6, " mi"
    elif ax >= 1e3:
        v, suf = x / 1e3, " mil"
    else:
        v, suf = x, ""
    s = f"R$ {v:,.1f}".replace(",", "X").replace(".", ",").replace("X", ".")
    return s + suf


def int_abbrev(x, pos=None):
    ax = abs(x)
    if ax >= 1e6:  return f"{x / 1e6:.1f}M"
    if ax >= 1e3:  return f"{x / 1e3:.1f}k"
    return f"{x:.0f}"


def brl_full(x: float) -> str:
    s = f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    return s


def _encode_png_base64(png_path: Path) -> str:
    data = png_path.read_bytes()
    return "data:image/png;base64," + base64.b64encode(data).decode("ascii")


canal = (
    df.groupby("Canal", dropna=True, as_index=False)
    .agg(
        receita=("Receita", "sum"),
        unidades=("Unidades_Vendidas", "sum")
    )
    .sort_values("receita", ascending=False)
)

total_receita = float(canal["receita"].sum()) if len(canal) else 0.0
canal["share_receita_%"] = (canal["receita"] / total_receita * 100.0) if total_receita else 0.0

canal_path = results_dir / "06_canal_mix.csv"
canal.to_csv(canal_path, index=False, encoding="utf-8")
print(f"[OK] Canal mix salvo em: {canal_path}")

canal_plot = canal.sort_values("receita", ascending=True)


fig1, ax1 = plt.subplots(figsize=(12, max(5.5, 0.45 * len(canal_plot))))
y = canal_plot["Canal"]
x = canal_plot["receita"]
bars1 = ax1.barh(y, x)
ax1.set_title("Receita por Canal")
ax1.set_xlabel("Receita (BRL)")
ax1.xaxis.set_major_formatter(mticker.FuncFormatter(brl_abbrev))
ax1.grid(True, axis="x", alpha=0.25)

labels1 = [f"{brl_abbrev(v)}  •  {s:.1f}%" for v, s in zip(x, canal_plot["share_receita_%"])]
try:
    ax1.bar_label(bars1, labels=labels1, padding=2)
except Exception:
    for rect, txt in zip(bars1, labels1):
        ax1.text(rect.get_width(), rect.get_y() + rect.get_height() / 2, " " + txt,
                 va="center", ha="left")
plt.tight_layout()
fig1_svg = results_dir / "06_canal_receita.svg"
fig1_png = results_dir / "06_canal_receita.png"
fig1.savefig(fig1_svg, bbox_inches="tight")
fig1.savefig(fig1_png, dpi=160, bbox_inches="tight")
plt.close(fig1)
print(f"[OK] Figuras: {fig1_svg} ; {fig1_png}")

fig2, ax2 = plt.subplots(figsize=(12, max(5.5, 0.45 * len(canal_plot))))
y2 = canal_plot["Canal"]
x2 = canal_plot["unidades"]
bars2 = ax2.barh(y2, x2)
ax2.set_title("Unidades por Canal")
ax2.set_xlabel("Unidades")
ax2.xaxis.set_major_formatter(mticker.FuncFormatter(int_abbrev))
ax2.grid(True, axis="x", alpha=0.25)
labels2 = [f"{int_abbrev(v)}" for v in x2]
try:
    ax2.bar_label(bars2, labels=labels2, padding=2)
except Exception:
    for rect, txt in zip(bars2, labels2):
        ax2.text(rect.get_width(), rect.get_y() + rect.get_height() / 2, " " + txt,
                 va="center", ha="left")
plt.tight_layout()
fig2_svg = results_dir / "06_canal_unidades.svg"
fig2_png = results_dir / "06_canal_unidades.png"
fig2.savefig(fig2_svg, bbox_inches="tight")
fig2.savefig(fig2_png, dpi=160, bbox_inches="tight")
plt.close(fig2)
print(f"[OK] Figuras: {fig2_svg} ; {fig2_png}")

share_top = canal.head(3)["share_receita_%"].sum() if len(canal) else 0.0
report_path = results_dir / "06_canal_report.html"
report_html = f"""<!doctype html>
<meta charset="utf-8">
<title>Mix por Canal — Desempenho</title>
<style>
body {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, "Helvetica Neue", Arial; margin: 24px; color: #111; }}
figure {{ margin: 0 0 28px 0; }}
figcaption {{ margin-top: 8px; font-size: 0.95rem; color: #444; }}
img {{ max-width: 100%; height: auto; border: 0; }}
p.code {{ font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; font-size: 0.95rem; }}
ul {{ line-height: 1.45; }}
hr {{ border: none; border-top: 1px solid #ddd; margin: 28px 0; }}
.note {{ background: #f6f6f6; border: 1px solid #e5e5e5; padding: 12px; border-radius: 8px; font-size: 0.95rem; }}
</style>

<h1>Mix por Canal</h1>
<div class="note">
  Abra este arquivo no mesmo diretório dos artefatos: <b>{fig1_svg.name}</b>, <b>{fig2_svg.name}</b> e <b>{canal_path.name}</b>.
</div>

<ul>
  <li>Número de canais: <b>{len(canal)}</b></li>
  <li>Receita total: <b>{brl_full(total_receita)}</b></li>
  <li>Top 3 canais concentram: <b>{share_top:.1f}%</b> da receita</li>
</ul>

<figure>
  <img src="{fig1_svg.name}" alt="Receita por Canal">
  <figcaption>Receita por Canal (valor + participação %).</figcaption>
</figure>

<figure>
  <img src="{fig2_svg.name}" alt="Unidades por Canal">
  <figcaption>Unidades por Canal (rótulos abreviados).</figcaption>
</figure>

<p class="code">CSV: {canal_path.name}</p>
"""
report_path.write_text(report_html, encoding="utf-8")
print(f"[OK] Relatório HTML (arquivos externos): {report_path}")

img1_data = _encode_png_base64(fig1_png)
img2_data = _encode_png_base64(fig2_png)

report_inline_path = results_dir / "06_canal_report_inline.html"
report_inline_html = f"""<!doctype html>
<meta charset="utf-8">
<title>Mix por Canal — (Inline)</title>
<style>
body {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, "Helvetica Neue", Arial; margin: 24px; color: #111; }}
figure {{ margin: 0 0 28px 0; }}
figcaption {{ margin-top: 8px; font-size: 0.95rem; color: #444; }}
img {{ max-width: 100%; height: auto; border: 0; }}
p.code {{ font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; font-size: 0.95rem; }}
ul {{ line-height: 1.45; }}
hr {{ border: none; border-top: 1px solid #ddd; margin: 28px 0; }}
</style>

<h1>Mix por Canal — Relatório Inline</h1>
<ul>
  <li>Número de canais: <b>{len(canal)}</b></li>
  <li>Receita total: <b>{brl_full(total_receita)}</b></li>
  <li>Top 3 canais concentram: <b>{share_top:.1f}%</b> da receita</li>
</ul>

<figure>
  <img src="{img1_data}" alt="Receita por Canal">
  <figcaption>Receita por Canal (valor + participação %).</figcaption>
</figure>

<figure>
  <img src="{img2_data}" alt="Unidades por Canal">
  <figcaption>Unidades por Canal (rótulos abreviados).</figcaption>
</figure>

<p class="code">CSV externo: {canal_path.name}</p>
"""
report_inline_path.write_text(report_inline_html, encoding="utf-8")
print(f"[OK] Relatório HTML INLINE (auto-contido): {report_inline_path}")

[OK] Canal mix salvo em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\06_canal_mix.csv
[OK] Figuras: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\06_canal_receita.svg ; C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\06_canal_receita.png
[OK] Figuras: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\06_canal_unidades.svg ; C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\06_canal_unidades.png
[OK] Relatório HTML (arquivos externos): C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\06_canal_report.html
[OK] Relatório HTML INLINE (auto-contido): C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\06_canal_report_inline.html


In [48]:
from pathlib import Path
import base64
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here


    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

assert "df" in locals(), "DataFrame 'df' não encontrado."
assert "Regiões" in df.columns, "Coluna 'Regiões' ausente."


def brl_abbrev(x, pos=None):
    ax = abs(x)
    if ax >= 1e9:
        v, suf = x / 1e9, " bi"
    elif ax >= 1e6:
        v, suf = x / 1e6, " mi"
    elif ax >= 1e3:
        v, suf = x / 1e3, " mil"
    else:
        v, suf = x, ""
    s = f"R$ {v:,.1f}".replace(",", "X").replace(".", ",").replace("X", ".")
    return s + suf


def int_abbrev(x, pos=None):
    ax = abs(x)
    if ax >= 1e6:  return f"{x / 1e6:.1f}M"
    if ax >= 1e3:  return f"{x / 1e3:.1f}k"
    return f"{x:.0f}"


def brl_full(x: float) -> str:
    s = f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    return s


def _encode_png_base64(png_path: Path) -> str:
    data = png_path.read_bytes()
    return "data:image/png;base64," + base64.b64encode(data).decode("ascii")


reg = (
    df.groupby("Regiões", dropna=True, as_index=False)
    .agg(
        receita=("Receita", "sum"),
        unidades=("Unidades_Vendidas", "sum"),
    )
    .sort_values("receita", ascending=False)
)

total_receita = float(reg["receita"].sum()) if len(reg) else 0.0
reg["share_receita_%"] = (reg["receita"] / total_receita * 100.0) if total_receita else 0.0

reg_path = results_dir / "07_regiao_distribuicao.csv"
reg.to_csv(reg_path, index=False, encoding="utf-8")
print(f"[OK] Distribuição regional salva em: {reg_path}")

reg_plot = reg.sort_values("receita", ascending=True)

fig1, ax1 = plt.subplots(figsize=(12, max(5.5, 0.45 * len(reg_plot))))
y = reg_plot["Regiões"]
x = reg_plot["receita"]
bars1 = ax1.barh(y, x)
ax1.set_title("Receita por Região")
ax1.set_xlabel("Receita (BRL)")
ax1.xaxis.set_major_formatter(mticker.FuncFormatter(brl_abbrev))
ax1.grid(True, axis="x", alpha=0.25)

labels1 = [f"{brl_abbrev(v)}  •  {s:.1f}%" for v, s in zip(x, reg_plot["share_receita_%"])]
try:
    ax1.bar_label(bars1, labels=labels1, padding=2)
except Exception:
    for rect, txt in zip(bars1, labels1):
        ax1.text(rect.get_width(), rect.get_y() + rect.get_height() / 2, " " + txt,
                 va="center", ha="left")
plt.tight_layout()
fig1_svg = results_dir / "07_regiao_receita.svg"
fig1_png = results_dir / "07_regiao_receita.png"
fig1.savefig(fig1_svg, bbox_inches="tight")
fig1.savefig(fig1_png, dpi=160, bbox_inches="tight")
plt.close(fig1)
print(f"[OK] Figuras: {fig1_svg} ; {fig1_png}")

fig2, ax2 = plt.subplots(figsize=(12, max(5.5, 0.45 * len(reg_plot))))
y2 = reg_plot["Regiões"]
x2 = reg_plot["unidades"]
bars2 = ax2.barh(y2, x2)
ax2.set_title("Unidades por Região")
ax2.set_xlabel("Unidades")
ax2.xaxis.set_major_formatter(mticker.FuncFormatter(int_abbrev))
ax2.grid(True, axis="x", alpha=0.25)
labels2 = [f"{int_abbrev(v)}" for v in x2]
try:
    ax2.bar_label(bars2, labels=labels2, padding=2)
except Exception:
    for rect, txt in zip(bars2, labels2):
        ax2.text(rect.get_width(), rect.get_y() + rect.get_height() / 2, " " + txt,
                 va="center", ha="left")
plt.tight_layout()
fig2_svg = results_dir / "07_regiao_unidades.svg"
fig2_png = results_dir / "07_regiao_unidades.png"
fig2.savefig(fig2_svg, bbox_inches="tight")
fig2.savefig(fig2_png, dpi=160, bbox_inches="tight")
plt.close(fig2)
print(f"[OK] Figuras: {fig2_svg} ; {fig2_png}")

share_top3 = reg.head(3)["share_receita_%"].sum() if len(reg) else 0.0
report_path = results_dir / "07_regiao_report.html"
report_html = f"""<!doctype html>
<meta charset="utf-8">
<title>Distribuição Regional — Desempenho</title>
<style>
body {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, "Helvetica Neue", Arial; margin: 24px; color: #111; }}
figure {{ margin: 0 0 28px 0; }}
figcaption {{ margin-top: 8px; font-size: 0.95rem; color: #444; }}
img {{ max-width: 100%; height: auto; border: 0; }}
p.code {{ font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; font-size: 0.95rem; }}
ul {{ line-height: 1.45; }}
hr {{ border: none; border-top: 1px solid #ddd; margin: 28px 0; }}
.note {{ background: #f6f6f6; border: 1px solid #e5e5e5; padding: 12px; border-radius: 8px; font-size: 0.95rem; }}
</style>

<h1>Distribuição Regional</h1>
<div class="note">
  Abra este arquivo no mesmo diretório dos artefatos: <b>{fig1_svg.name}</b>, <b>{fig2_svg.name}</b> e <b>{reg_path.name}</b>.
</div>

<ul>
  <li>Número de regiões: <b>{len(reg)}</b></li>
  <li>Receita total: <b>{brl_full(total_receita)}</b></li>
  <li>Top 3 regiões concentram: <b>{share_top3:.1f}%</b> da receita</li>
</ul>

<figure>
  <img src="{fig1_svg.name}" alt="Receita por Região">
  <figcaption>Receita por Região (valor + participação %).</figcaption>
</figure>

<figure>
  <img src="{fig2_svg.name}" alt="Unidades por Região">
  <figcaption>Unidades por Região (rótulos abreviados).</figcaption>
</figure>

<p class="code">CSV: {reg_path.name}</p>
"""
report_path.write_text(report_html, encoding="utf-8")
print(f"[OK] Relatório HTML (arquivos externos): {report_path}")

img1_data = _encode_png_base64(fig1_png)
img2_data = _encode_png_base64(fig2_png)

report_inline_path = results_dir / "07_regiao_report_inline.html"
report_inline_html = f"""<!doctype html>
<meta charset="utf-8">
<title>Distribuição Regional — (Inline)</title>
<style>
body {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, "Helvetica Neue", Arial; margin: 24px; color: #111; }}
figure {{ margin: 0 0 28px 0; }}
figcaption {{ margin-top: 8px; font-size: 0.95rem; color: #444; }}
img {{ max-width: 100%; height: auto; border: 0; }}
p.code {{ font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; font-size: 0.95rem; }}
ul {{ line-height: 1.45; }}
hr {{ border: none; border-top: 1px solid #ddd; margin: 28px 0; }}
</style>

<h1>Distribuição Regional — Relatório Inline</h1>
<ul>
  <li>Número de regiões: <b>{len(reg)}</b></li>
  <li>Receita total: <b>{brl_full(total_receita)}</b></li>
  <li>Top 3 regiões concentram: <b>{share_top3:.1f}%</b> da receita</li>
</ul>

<figure>
  <img src="{img1_data}" alt="Receita por Região">
  <figcaption>Receita por Região (valor + participação %).</figcaption>
</figure>

<figure>
  <img src="{img2_data}" alt="Unidades por Região">
  <figcaption>Unidades por Região (rótulos abreviados).</figcaption>
</figure>

<p class="code">CSV externo: {reg_path.name}</p>
"""
report_inline_path.write_text(report_inline_html, encoding="utf-8")
print(f"[OK] Relatório HTML INLINE (auto-contido): {report_inline_path}")

[OK] Distribuição regional salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\07_regiao_distribuicao.csv
[OK] Figuras: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\07_regiao_receita.svg ; C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\07_regiao_receita.png
[OK] Figuras: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\07_regiao_unidades.svg ; C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\07_regiao_unidades.png
[OK] Relatório HTML (arquivos externos): C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\07_regiao_report.html
[OK] Relatório HTML INLINE (auto-contido): C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\07_regiao_report_inline.html


In [49]:
from pathlib import Path
import base64, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here


    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

assert "df" in locals(), "DataFrame 'df' não encontrado."
assert {"Preço_Unitário", "Unidades_Vendidas"} <= set(df.columns), "Faltam colunas necessárias."


def brl_abbrev(x, pos=None):
    ax = abs(x)
    if ax >= 1e9:
        v, suf = x / 1e9, " bi"
    elif ax >= 1e6:
        v, suf = x / 1e6, " mi"
    elif ax >= 1e3:
        v, suf = x / 1e3, " mil"
    else:
        v, suf = x, ""
    s = f"R$ {v:,.1f}".replace(",", "X").replace(".", ",").replace("X", ".")
    return s + suf


def int_abbrev(x, pos=None):
    ax = abs(x)
    if ax >= 1e6:  return f"{x / 1e6:.1f}M"
    if ax >= 1e3:  return f"{x / 1e3:.1f}k"
    return f"{x:.0f}"


def brl_full(x: float) -> str:
    s = f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    return s


def _encode_png_base64(png_path: Path) -> str:
    data = png_path.read_bytes()
    return "data:image/png;base64," + base64.b64encode(data).decode("ascii")


aux = (
    df[["Preço_Unitário", "Unidades_Vendidas"]]
    .copy()
)

for col in aux.columns:
    aux[col] = pd.to_numeric(aux[col], errors="coerce")
aux = aux.dropna()


corr_pearson = aux.corr(method="pearson")
corr_spearman = aux.corr(method="spearman")

corr_table = pd.DataFrame({
    "metric": ["pearson_r", "spearman_rho"],
    "coef": [
        float(corr_pearson.loc["Preço_Unitário", "Unidades_Vendidas"])
        if "Preço_Unitário" in corr_pearson.index and "Unidades_Vendidas" in corr_pearson.columns else float("nan"),
        float(corr_spearman.loc["Preço_Unitário", "Unidades_Vendidas"])
        if "Preço_Unitário" in corr_spearman.index and "Unidades_Vendidas" in corr_spearman.columns else float("nan"),
    ]
})
corr_path = results_dir / "08_correlacao_preco_unidades.csv"
corr_table.to_csv(corr_path, index=False, encoding="utf-8")
print(f"[OK] Correlações salvas em: {corr_path}")


x = aux["Preço_Unitário"].to_numpy()
y = aux["Unidades_Vendidas"].to_numpy()
N = len(aux)


def _should_log(v: np.ndarray) -> bool:
    v = v[np.isfinite(v) & (v > 0)]
    if len(v) < 2: return False
    r = (v.max() / max(v.min(), 1e-12))
    return r >= 100


x_log = _should_log(x)
y_log = _should_log(y)

ols_coef = np.polyfit(x, y, 1) if N >= 2 else np.array([np.nan, np.nan])
ols_slope, ols_intercept = float(ols_coef[0]), float(ols_coef[1])

robust_available = False
try:
    from sklearn.linear_model import TheilSenRegressor

    ts = TheilSenRegressor(random_state=0)
    ts.fit(x.reshape(-1, 1), y)
    ts_slope = float(ts.coef_[0])
    ts_intercept = float(ts.intercept_)
    robust_available = True
except Exception:
    ts_slope = ts_intercept = float("nan")

USE_HEXBIN = N >= 5000

fig, ax = plt.subplots(figsize=(12, 7))

if USE_HEXBIN:
    hb = ax.hexbin(x, y, gridsize=40, bins="log")
    cbar = fig.colorbar(hb, ax=ax)
    cbar.set_label("Densidade (log)")
    kind = "hexbin"
else:
    ax.scatter(x, y, s=16, alpha=0.35)
    kind = "scatter"

xline = np.linspace(np.nanmin(x), np.nanmax(x), 200)
y_ols = ols_slope * xline + ols_intercept
ax.plot(xline, y_ols, linestyle="--", linewidth=2,
        label=f"OLS: y = {ols_slope:.3g}·x + {ols_intercept:.3g}")

if robust_available:
    y_ts = ts_slope * xline + ts_intercept
    ax.plot(xline, y_ts, linewidth=2,
            label=f"Theil-Sen: y = {ts_slope:.3g}·x + {ts_intercept:.3g}")

ax.set_title("Preço Unitário × Unidades Vendidas — Dispersão e Tendência")
ax.set_xlabel("Preço Unitário (BRL)")
ax.set_ylabel("Unidades Vendidas")
ax.grid(True, alpha=0.25)

ax.xaxis.set_major_formatter(mticker.FuncFormatter(brl_abbrev))
ax.yaxis.set_major_formatter(mticker.FuncFormatter(int_abbrev))

used_log = False
if x_log:
    ax.set_xscale("log")
    used_log = True
if y_log:
    ax.set_yscale("log")
    used_log = True

ax.legend(loc="best")
plt.tight_layout()

fig_svg = results_dir / "08_preco_unidades.svg"
fig_png = results_dir / "08_preco_unidades.png"
fig.savefig(fig_svg, bbox_inches="tight")
fig.savefig(fig_png, dpi=160, bbox_inches="tight")
plt.close(fig)
print(f"[OK] Figura ({kind}): {fig_svg} ; {fig_png}")

report_path = results_dir / "08_preco_unidades_report.html"
report_inline_path = results_dir / "08_preco_unidades_report_inline.html"

pearson_r = float(corr_table.loc[corr_table["metric"] == "pearson_r", "coef"].iloc[0])
spearman_rho = float(corr_table.loc[corr_table["metric"] == "spearman_rho", "coef"].iloc[0])
ols_eq = f"y = {ols_slope:.4g}·x + {ols_intercept:.4g}"
ts_eq = f"y = {ts_slope:.4g}·x + {ts_intercept:.4g}" if robust_available else "n/d"

html_style = """
<style>
body { font-family: system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, "Helvetica Neue", Arial; margin: 24px; color: #111; }
figure { margin: 0 0 28px 0; }
figcaption { margin-top: 8px; font-size: 0.95rem; color: #444; }
img { max-width: 100%; height: auto; border: 0; }
p.code { font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; font-size: 0.95rem; }
ul { line-height: 1.45; }
hr { border: none; border-top: 1px solid #ddd; margin: 28px 0; }
.note { background: #f6f6f6; border: 1px solid #e5e5e5; padding: 12px; border-radius: 8px; font-size: 0.95rem; }
.kv td:first-child { color: #444; padding-right: 10px; white-space: nowrap; }
table.kv { border-collapse: collapse; }
</style>
"""

report_html = f"""<!doctype html>
<meta charset="utf-8">
<title>Preço × Unidades — Dispersão e Correlação</title>
{html_style}
<h1>Preço × Unidades — Dispersão e Correlação</h1>
<div class="note">
  Artefatos no diretório: <b>{fig_svg.name}</b>, <b>{fig_png.name}</b>, <b>{corr_path.name}</b>.
</div>

<table class="kv">
  <tr><td>N amostras</td><td><b>{N:,}</b></td></tr>
  <tr><td>Pearson r</td><td><b>{pearson_r:.4f}</b></td></tr>
  <tr><td>Spearman \u03C1</td><td><b>{spearman_rho:.4f}</b></td></tr>
  <tr><td>OLS</td><td><b>{ols_eq}</b></td></tr>
  <tr><td>Theil-Sen</td><td><b>{ts_eq}</b></td></tr>
  <tr><td>Escala log usada?</td><td><b>{"Sim" if used_log else "Não"}</b></td></tr>
  <tr><td>Render</td><td><b>{kind}</b></td></tr>
</table>

<figure>
  <img src="{fig_svg.name}" alt="Dispersão Preço × Unidades">
  <figcaption>Dispersão com linha(s) de tendência; formatação de eixos e grade leve.</figcaption>
</figure>

<p class="code">CSV: {corr_path.name}</p>
"""
report_path.write_text(report_html, encoding="utf-8")
print(f"[OK] Relatório HTML: {report_path}")

img_data = _encode_png_base64(fig_png)
report_inline_html = f"""<!doctype html>
<meta charset="utf-8">
<title>Preço × Unidades — (Inline)</title>
{html_style}
<h1>Preço × Unidades — Dispersão e Correlação (Inline)</h1>

<table class="kv">
  <tr><td>N amostras</td><td><b>{N:,}</b></td></tr>
  <tr><td>Pearson r</td><td><b>{pearson_r:.4f}</b></td></tr>
  <tr><td>Spearman \u03C1</td><td><b>{spearman_rho:.4f}</b></td></tr>
  <tr><td>OLS</td><td><b>{ols_eq}</b></td></tr>
  <tr><td>Theil-Sen</td><td><b>{ts_eq}</b></td></tr>
  <tr><td>Escala log usada?</td><td><b>{"Sim" if used_log else "Não"}</b></td></tr>
  <tr><td>Render</td><td><b>{kind}</b></td></tr>
</table>

<figure>
  <img src="{img_data}" alt="Dispersão Preço × Unidades">
  <figcaption>Figura embutida (base64). CSV permanece externo: {corr_path.name}</figcaption>
</figure>
"""
report_inline_path.write_text(report_inline_html, encoding="utf-8")
print(f"[OK] Relatório HTML INLINE: {report_inline_path}")

[OK] Correlações salvas em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\08_correlacao_preco_unidades.csv
[OK] Figura (scatter): C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\08_preco_unidades.svg ; C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\08_preco_unidades.png
[OK] Relatório HTML: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\08_preco_unidades_report.html
[OK] Relatório HTML INLINE: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\08_preco_unidades_report_inline.html


In [50]:
from pathlib import Path
import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here


    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

assert "df" in locals(), "DataFrame 'df' não encontrado."

faixas = [
    "Faixa Etária 25 a 40 anos",
    "Faixa Etária 30 a 45 anos",
    "Faixa Etária 30+ anos",
    "Faixa Etária 35+ anos",
]
present = [c for c in faixas if c in df.columns]
assert len(present) > 0, f"Nenhuma das colunas de faixas etárias foi encontrada. Esperadas: {faixas}"


def int_abbrev(x, pos=None):
    ax = abs(x)
    if ax >= 1e6:  return f"{x / 1e6:.1f}M"
    if ax >= 1e3:  return f"{x / 1e3:.1f}k"
    return f"{x:.0f}"


def _encode_png_base64(png_path: Path) -> str:
    data = png_path.read_bytes()
    return "data:image/png;base64," + base64.b64encode(data).decode("ascii")


aux = df[present].apply(pd.to_numeric, errors="coerce")
agg = aux.sum(axis=0, skipna=True)
agg = agg.rename_axis("faixa").reset_index(name="unidades")

total = float(agg["unidades"].sum()) if len(agg) else 0.0
agg["participacao_%"] = (agg["unidades"] / total * 100.0) if total else np.nan

out_path = results_dir / "09_faixas_etarias.csv"
agg.to_csv(out_path, index=False, encoding="utf-8")
print(f"[OK] Tabela de faixas etárias salva em: {out_path}")

agg_plot = agg.sort_values("unidades", ascending=True)

fig, ax = plt.subplots(figsize=(12, max(5.5, 0.45 * len(agg_plot))))
y = agg_plot["faixa"]
x = agg_plot["unidades"]
bars = ax.barh(y, x)
ax.set_title("Unidades por Faixa Etária")
ax.set_xlabel("Unidades")
ax.xaxis.set_major_formatter(mticker.FuncFormatter(int_abbrev))
ax.grid(True, axis="x", alpha=0.25)

labels = [f"{int_abbrev(v)}  •  {s:.1f}%" for v, s in zip(x, agg_plot["participacao_%"])]
try:
    ax.bar_label(bars, labels=labels, padding=2)
except Exception:
    for rect, txt in zip(bars, labels):
        ax.text(rect.get_width(), rect.get_y() + rect.get_height() / 2, " " + txt,
                va="center", ha="left")
plt.tight_layout()

fig_svg = results_dir / "09_faixas_etarias_barras.svg"
fig_png = results_dir / "09_faixas_etarias_barras.png"
fig.savefig(fig_svg, bbox_inches="tight")
fig.savefig(fig_png, dpi=160, bbox_inches="tight")
plt.close(fig)
print(f"[OK] Figuras: {fig_svg} ; {fig_png}")

report_path = results_dir / "09_faixas_etarias_report.html"
html = f"""<!doctype html>
<meta charset="utf-8">
<title>Faixas Etárias — Unidades e Participação</title>
<style>
body {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, "Helvetica Neue", Arial; margin: 24px; color: #111; }}
figure {{ margin: 0 0 28px 0; }}
figcaption {{ margin-top: 8px; font-size: 0.95rem; color: #444; }}
img {{ max-width: 100%; height: auto; border: 0; }}
p.code {{ font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; font-size: 0.95rem; }}
ul {{ line-height: 1.45; }}
.note {{ background: #f6f6f6; border: 1px solid #e5e5e5; padding: 12px; border-radius: 8px; font-size: 0.95rem; }}
</style>

<h1>Faixas Etárias — Unidades e Participação</h1>
<div class="note">
  Observação: as faixas podem se sobrepor (ex.: 30+ e 35+). Se as colunas representarem populações não exclusivas, a soma total pode refletir
  contagens sobrepostas. Interprete a participação (%) como relativa ao total agregado destas colunas.
</div>

<ul>
  <li>Número de faixas presentes: <b>{len(agg)}</b></li>
  <li>Unidades totais (agregadas): <b>{int(agg['unidades'].sum()):,}</b></li>
</ul>

<figure>
  <img src="{fig_svg.name}" alt="Unidades por Faixa Etária">
  <figcaption>Barras horizontais com rótulos (unidades + participação %).</figcaption>
</figure>

<p class="code">CSV: {out_path.name}</p>
"""
report_path.write_text(html, encoding="utf-8")
print(f"[OK] Relatório HTML: {report_path}")

report_inline_path = results_dir / "09_faixas_etarias_report_inline.html"
img_data = _encode_png_base64(fig_png)
html_inline = f"""<!doctype html>
<meta charset="utf-8">
<title>Faixas Etárias — (Inline)</title>
<style>
body {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, "Helvetica Neue", Arial; margin: 24px; color: #111; }}
figure {{ margin: 0 0 28px 0; }}
figcaption {{ margin-top: 8px; font-size: 0.95rem; color: #444; }}
img {{ max-width: 100%; height: auto; border: 0; }}
p.code {{ font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; font-size: 0.95rem; }}
ul {{ line-height: 1.45; }}
.note {{ background: #f6f6f6; border: 1px solid #e5e5e5; padding: 12px; border-radius: 8px; font-size: 0.95rem; }}
</style>

<h1>Faixas Etárias — Relatório Inline</h1>
<div class="note">
  Observação: as faixas podem se sobrepor (ex.: 30+ e 35+). Se as colunas representarem populações não exclusivas, a soma total pode refletir
  contagens sobrepostas. Interprete a participação (%) como relativa ao total agregado destas colunas.
</div>

<ul>
  <li>Número de faixas presentes: <b>{len(agg)}</b></li>
  <li>Unidades totais (agregadas): <b>{int(agg['unidades'].sum()):,}</b></li>
</ul>

<figure>
  <img src="{img_data}" alt="Unidades por Faixa Etária">
  <figcaption>Figura embutida (base64). CSV permanece externo: {out_path.name}</figcaption>
</figure>
"""
report_inline_path.write_text(html_inline, encoding="utf-8")
print(f"[OK] Relatório HTML INLINE: {report_inline_path}")

agg

[OK] Tabela de faixas etárias salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\09_faixas_etarias.csv
[OK] Figuras: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\09_faixas_etarias_barras.svg ; C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\09_faixas_etarias_barras.png
[OK] Relatório HTML: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\09_faixas_etarias_report.html
[OK] Relatório HTML INLINE: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\09_faixas_etarias_report_inline.html


,faixa,unidades,participacao_%
0,Faixa Etária 25 a 40 anos,40019,30.000825
1,Faixa Etária 30 a 45 anos,46687,34.999588
2,Faixa Etária 30+ anos,26665,19.989805
3,Faixa Etária 35+ anos,20022,15.009783


In [51]:
from pathlib import Path
import base64, math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here


    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

assert "df" in locals(), "DataFrame 'df' não encontrado."


def brl_abbrev(x, pos=None):
    ax = abs(x)
    if ax >= 1e9:
        v, suf = x / 1e9, " bi"
    elif ax >= 1e6:
        v, suf = x / 1e6, " mi"
    elif ax >= 1e3:
        v, suf = x / 1e3, " mil"
    else:
        v, suf = x, ""
    s = f"R$ {v:,.1f}".replace(",", "X").replace(".", ",").replace("X", ".")
    return s + suf


def int_abbrev(x, pos=None):
    ax = abs(x)
    if ax >= 1e6:  return f"{x / 1e6:.1f}M"
    if ax >= 1e3:  return f"{x / 1e3:.1f}k"
    return f"{x:.0f}"


def brl_full(x: float) -> str:
    s = f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    return s


def _encode_png_base64(png_path: Path) -> str:
    data = png_path.read_bytes()
    return "data:image/png;base64," + base64.b64encode(data).decode("ascii")


def _top_n_com_outros(df_ranked: pd.DataFrame, top_n: int, chave_col: str) -> pd.DataFrame:
    """Retorna Top-N por 'receita' + linha 'Outros' (somando o resto)."""
    top = df_ranked.nlargest(top_n, "receita").copy()
    if len(df_ranked) > top_n:
        resto = df_ranked.iloc[top_n:]
        outros = pd.DataFrame({
            chave_col: ["Outros"],
            "receita": [resto["receita"].sum()],
            "unidades": [resto["unidades"].sum()],
            "share_receita_%": [resto["share_receita_%"].sum()],
        })
        out = pd.concat([top, outros], ignore_index=True)
    else:
        out = top
    return out.sort_values("receita", ascending=True).reset_index(drop=True)


def _plot_barh_val_share(df_plot: pd.DataFrame, y_col: str, x_col: str, titulo: str, xlabel: str, formater):
    fig, ax = plt.subplots(figsize=(12, max(6, 0.45 * len(df_plot))))
    y = df_plot[y_col]
    x = df_plot[x_col]
    bars = ax.barh(y, x)
    ax.set_title(titulo)
    ax.set_xlabel(xlabel)
    ax.xaxis.set_major_formatter(mticker.FuncFormatter(formater))
    ax.grid(True, axis="x", alpha=0.25)
    if "share_receita_%":
        if x_col == "receita":
            labels = [f"{brl_abbrev(v)}  •  {s:.1f}%" for v, s in zip(x, df_plot["share_receita_%"])]
        else:
            labels = [f"{int_abbrev(v)}" for v in x]
    try:
        ax.bar_label(bars, labels=labels, padding=2)
    except Exception:
        for rect, txt in zip(bars, labels):
            ax.text(rect.get_width(), rect.get_y() + rect.get_height() / 2, " " + txt,
                    va="center", ha="left")
    plt.tight_layout()
    return fig


TOP_N = 15

figs = []
paths_png = []
paths_svg = []

if "Loja" in df.columns:
    loja = (
        df.groupby("Loja", dropna=True, as_index=False)
        .agg(receita=("Receita", "sum"),
             unidades=("Unidades_Vendidas", "sum"))
        .sort_values("receita", ascending=False)
    )
    total_receita_loja = float(loja["receita"].sum()) if len(loja) else 0.0
    loja["share_receita_%"] = (loja["receita"] / total_receita_loja * 100.0) if total_receita_loja else 0.0

    loja_path = results_dir / "10_loja_desempenho.csv"
    loja.to_csv(loja_path, index=False, encoding="utf-8")
    print(f"[OK] Loja desempenho salvo em: {loja_path}")

    loja_plot = _top_n_com_outros(loja, TOP_N, "Loja")

    fig_l1 = _plot_barh_val_share(loja_plot, "Loja", "receita",
                                  f"Receita por Loja — Top {min(TOP_N, len(loja))} (+ Outros)",
                                  "Receita (BRL)", brl_abbrev)
    fig_l1_svg = results_dir / "10_loja_receita_top.svg"
    fig_l1_png = results_dir / "10_loja_receita_top.png"
    fig_l1.savefig(fig_l1_svg, bbox_inches="tight")
    fig_l1.savefig(fig_l1_png, dpi=160, bbox_inches="tight")
    plt.close(fig_l1)
    figs.append(fig_l1_png)
    paths_png.append(fig_l1_png)
    paths_svg.append(fig_l1_svg)
    print(f"[OK] Figuras: {fig_l1_svg} ; {fig_l1_png}")

    fig_l2 = _plot_barh_val_share(loja_plot, "Loja", "unidades",
                                  f"Unidades por Loja — Top {min(TOP_N, len(loja))} (+ Outros)",
                                  "Unidades", int_abbrev)
    fig_l2_svg = results_dir / "10_loja_unidades_top.svg"
    fig_l2_png = results_dir / "10_loja_unidades_top.png"
    fig_l2.savefig(fig_l2_svg, bbox_inches="tight")
    fig_l2.savefig(fig_l2_png, dpi=160, bbox_inches="tight")
    plt.close(fig_l2)
    figs.append(fig_l2_png)
    paths_png.append(fig_l2_png)
    paths_svg.append(fig_l2_svg)
    print(f"[OK] Figuras: {fig_l2_svg} ; {fig_l2_png}")
else:
    print("[WARN] Coluna 'Loja' ausente.")

if "Funcionario" in df.columns:
    func = (
        df.groupby("Funcionario", dropna=True, as_index=False)
        .agg(receita=("Receita", "sum"),
             unidades=("Unidades_Vendidas", "sum"))
        .sort_values("receita", ascending=False)
    )
    total_receita_func = float(func["receita"].sum()) if len(func) else 0.0
    func["share_receita_%"] = (func["receita"] / total_receita_func * 100.0) if total_receita_func else 0.0

    func_path = results_dir / "10_funcionario_desempenho.csv"
    func.to_csv(func_path, index=False, encoding="utf-8")
    print(f"[OK] Funcionário desempenho salvo em: {func_path}")

    func_plot = _top_n_com_outros(func, TOP_N, "Funcionario")

    fig_f1 = _plot_barh_val_share(func_plot, "Funcionario", "receita",
                                  f"Receita por Funcionário — Top {min(TOP_N, len(func))} (+ Outros)",
                                  "Receita (BRL)", brl_abbrev)
    fig_f1_svg = results_dir / "10_funcionario_receita_top.svg"
    fig_f1_png = results_dir / "10_funcionario_receita_top.png"
    fig_f1.savefig(fig_f1_svg, bbox_inches="tight")
    fig_f1.savefig(fig_f1_png, dpi=160, bbox_inches="tight")
    plt.close(fig_f1)
    figs.append(fig_f1_png)
    paths_png.append(fig_f1_png)
    paths_svg.append(fig_f1_svg)
    print(f"[OK] Figuras: {fig_f1_svg} ; {fig_f1_png}")

    fig_f2 = _plot_barh_val_share(func_plot, "Funcionario", "unidades",
                                  f"Unidades por Funcionário — Top {min(TOP_N, len(func))} (+ Outros)",
                                  "Unidades", int_abbrev)
    fig_f2_svg = results_dir / "10_funcionario_unidades_top.svg"
    fig_f2_png = results_dir / "10_funcionario_unidades_top.png"
    fig_f2.savefig(fig_f2_svg, bbox_inches="tight")
    fig_f2.savefig(fig_f2_png, dpi=160, bbox_inches="tight")
    plt.close(fig_f2)
    figs.append(fig_f2_png)
    paths_png.append(fig_f2_png)
    paths_svg.append(fig_f2_svg)
    print(f"[OK] Figuras: {fig_f2_svg} ; {fig_f2_png}")
else:
    print("[WARN] Coluna 'Funcionario' ausente.")

report_path = results_dir / "10_loja_func_report.html"
html_style = """
<style>
body { font-family: system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, "Helvetica Neue", Arial; margin: 24px; color: #111; }
figure { margin: 0 0 28px 0; }
figcaption { margin-top: 8px; font-size: 0.95rem; color: #444; }
img { max-width: 100%; height: auto; border: 0; }
p.code { font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; font-size: 0.95rem; }
ul { line-height: 1.45; }
hr { border: none; border-top: 1px solid #ddd; margin: 28px 0; }
.note { background: #f6f6f6; border: 1px solid #e5e5e5; padding: 12px; border-radius: 8px; font-size: 0.95rem; }
</style>
"""
blocks = []
if "Loja" in df.columns:
    blocks.append(f"""
<h2>Loja</h2>
<ul>
  <li>Receita total (Lojas): <b>{brl_full(total_receita_loja)}</b></li>
  <li>Top {min(TOP_N, len(loja))} agregado + 'Outros'</li>
</ul>
<figure><img src="{paths_svg[0].name}" alt="Receita por Loja — Top + Outros"><figcaption>Receita por Loja — Top + Outros.</figcaption></figure>
<figure><img src="{paths_svg[1].name}" alt="Unidades por Loja — Top + Outros"><figcaption>Unidades por Loja — Top + Outros.</figcaption></figure>
<p class="code">CSV:  {results_dir.name}/10_loja_desempenho.csv</p>
""")
if "Funcionario" in df.columns:
    start = 2 if "Loja" in df.columns else 0
    blocks.append(f"""
<h2>Funcionário</h2>
<ul>
  <li>Receita total (Funcionários): <b>{brl_full(total_receita_func)}</b></li>
  <li>Top {min(TOP_N, len(func))} agregado + 'Outros'</li>
</ul>
<figure><img src="{paths_svg[start].name}" alt="Receita por Funcionário — Top + Outros"><figcaption>Receita por Funcionário — Top + Outros.</figcaption></figure>
<figure><img src="{paths_svg[start + 1].name}" alt="Unidades por Funcionário — Top + Outros"><figcaption>Unidades por Funcionário — Top + Outros.</figcaption></figure>
<p class="code">CSV:  {results_dir.name}/10_funcionario_desempenho.csv</p>
""")

report_html = f"""<!doctype html>
<meta charset="utf-8">
<title>Desempenho — Lojas e Funcionários</title>
{html_style}
<h1>Desempenho — Lojas e Funcionários</h1>
<div class="note">Abra este arquivo no mesmo diretório dos artefatos (SVG/PNG/CSV) gerados acima.</div>
{''.join(blocks)}
"""
report_path.write_text(report_html, encoding="utf-8")
print(f"[OK] Relatório HTML: {report_path}")

report_inline_path = results_dir / "10_loja_func_report_inline.html"
imgs_inline = [_encode_png_base64(p) for p in paths_png]
blocks_inline = []
idx = 0
if "Loja" in df.columns:
    blocks_inline.append(f"""
<h2>Loja</h2>
<ul>
  <li>Receita total (Lojas): <b>{brl_full(total_receita_loja)}</b></li>
  <li>Top {min(TOP_N, len(loja))} agregado + 'Outros'</li>
</ul>
<figure><img src="{imgs_inline[idx]}" alt="Receita por Loja — Top + Outros"><figcaption>Receita por Loja — Top + Outros.</figcaption></figure>
<figure><img src="{imgs_inline[idx + 1]}" alt="Unidades por Loja — Top + Outros"><figcaption>Unidades por Loja — Top + Outros.</figcaption></figure>
<p class="code">CSV externo:  {results_dir.name}/10_loja_desempenho.csv</p>
""")
    idx += 2
if "Funcionario" in df.columns:
    blocks_inline.append(f"""
<h2>Funcionário</h2>
<ul>
  <li>Receita total (Funcionários): <b>{brl_full(total_receita_func)}</b></li>
  <li>Top {min(TOP_N, len(func))} agregado + 'Outros'</li>
</ul>
<figure><img src="{imgs_inline[idx]}" alt="Receita por Funcionário — Top + Outros"><figcaption>Receita por Funcionário — Top + Outros.</figcaption></figure>
<figure><img src="{imgs_inline[idx + 1]}" alt="Unidades por Funcionário — Top + Outros"><figcaption>Unidades por Funcionário — Top + Outros.</figcaption></figure>
<p class="code">CSV externo:  {results_dir.name}/10_funcionario_desempenho.csv</p>
""")

report_inline_html = f"""<!doctype html>
<meta charset="utf-8">
<title>Desempenho — Lojas e Funcionários (Inline)</title>
{html_style}
<h1>Desempenho — Lojas e Funcionários (Inline)</h1>
{''.join(blocks_inline)}
"""
report_inline_path.write_text(report_inline_html, encoding="utf-8")
print(f"[OK] Relatório HTML INLINE: {report_inline_path}")

[OK] Loja desempenho salvo em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\10_loja_desempenho.csv
[OK] Figuras: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\10_loja_receita_top.svg ; C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\10_loja_receita_top.png
[OK] Figuras: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\10_loja_unidades_top.svg ; C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\10_loja_unidades_top.png
[OK] Funcionário desempenho salvo em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\10_funcionario_desempenho.csv
[OK] Figuras: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\10_funcionario_receita_top.svg ; C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\10_funcionario_receita_top.png
[OK] Figuras: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\10_funcionario_unidades_top.svg ; C:\Users\V